In [1]:
import torch.nn as nn
import torch
from torch.utils.data import TensorDataset, DataLoader
from skimage.measure import find_contours
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# Mi libreria:
from processLIDC import Patient

In [3]:
# vuelvo a instanciar:
patient = Patient("LIDC-IDRI-0013")
patient.scale()

Loading dicom files ... This may take a moment.


Importamos el modelo del cerebro preenrenado:

In [4]:
model = torch.hub.load('mateuszbuda/brain-segmentation-pytorch', 'unet',
                       in_channels=3, out_channels=1, init_features=32, pretrained=True)

Using cache found in /home/faraujo/.cache/torch/hub/mateuszbuda_brain-segmentation-pytorch_master


## Entrenar

In [5]:
## Metodos y atributos del modelo de pytorch:
ll = ['T_destination', '__annotations__', '__call__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_apply', '_backward_hooks', '_block', '_buffers', '_call_impl', '_forward_hooks', '_forward_pre_hooks', '_get_backward_hooks', '_get_name', '_is_full_backward_hook', '_load_from_state_dict', '_load_state_dict_pre_hooks', '_maybe_warn_non_full_backward_hook', '_modules', '_named_members', '_non_persistent_buffers_set', '_parameters', '_register_load_state_dict_pre_hook', '_register_state_dict_hook', '_replicate_for_data_parallel', '_save_to_state_dict', '_slow_forward', '_state_dict_hooks', '_version', 'add_module', 'apply', 'bfloat16', 'bottleneck', 'buffers', 'children', 'conv', 'cpu', 'cuda', 'decoder1', 'decoder2', 'decoder3', 'decoder4', 'double', 'dump_patches', 'encoder1', 'encoder2', 'encoder3', 'encoder4', 'eval', 'extra_repr', 'float', 'forward', 'get_buffer', 'get_extra_state', 'get_parameter', 'get_submodule', 'half', 'load_state_dict', 'modules', 'named_buffers', 'named_children', 'named_modules', 'named_parameters', 'parameters', 'pool1', 'pool2', 'pool3', 'pool4', 'register_backward_hook', 'register_buffer', 'register_forward_hook', 'register_forward_pre_hook', 'register_full_backward_hook', 'register_module', 'register_parameter', 'requires_grad_', 'set_extra_state', 'share_memory', 'state_dict', 'to', 'to_empty', 'train', 'training', 'type', 'upconv1', 'upconv2', 'upconv3', 'upconv4', 'xpu', 'zero_grad']
for l in ll:
    if not '_' == l[0]:
        print(l)

T_destination
add_module
apply
bfloat16
bottleneck
buffers
children
conv
cpu
cuda
decoder1
decoder2
decoder3
decoder4
double
dump_patches
encoder1
encoder2
encoder3
encoder4
eval
extra_repr
float
forward
get_buffer
get_extra_state
get_parameter
get_submodule
half
load_state_dict
modules
named_buffers
named_children
named_modules
named_parameters
parameters
pool1
pool2
pool3
pool4
register_backward_hook
register_buffer
register_forward_hook
register_forward_pre_hook
register_full_backward_hook
register_module
register_parameter
requires_grad_
set_extra_state
share_memory
state_dict
to
to_empty
train
training
type
upconv1
upconv2
upconv3
upconv4
xpu
zero_grad


In [7]:
imgs, mask = patient.get_tensors(scaled=True)

In [8]:
# Definir función de pérdida
loss_fn = nn.BCELoss()

# Definir optimizador
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
n_slices = 10
primera = 88-n_slices
ultima = 88+n_slices
dataset = TensorDataset(imgs[primera:ultima], mask[primera:ultima])

# Lista para almacenar la pérdida de entrenamiento
train_loss_history = []

# Crear el train_loader
batch_size = 4
train_loader = DataLoader(dataset,batch_size=9, shuffle=True)
for epoch in range(20):

    for batch_idx, (data, target) in enumerate(train_loader):
        # print(data.shape, target.shape)
        # # Forward pass
        output = model(data)
        # Calcular pérdida
        # plt.imshow(target[0])
        # plt.show()
        # print(data.shape, target.shape, output.shape)
        loss = loss_fn(output[:,0], target)

        # # # Calcular gradientes y actualizar parámetros
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_loss_history.append(loss.item())

        # Imprimir progreso
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


Train Epoch: 0 [0/20 (0%)]	Loss: 3.161296


KeyboardInterrupt: 

In [ ]:
plt.plot(np.log(np.array(train_loss_history)), label='Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

## Probemoslo:

In [ ]:
imgs, mask = patient.get_tensors(scaled=True)

In [ ]:
# slices:
# primera = 164
# ultima = 189

# Predecir:
pred = model(imgs[primera:ultima])
pred = torch.round(pred).detach().numpy()

In [ ]:
imgs[primera:ultima].shape

In [ ]:
for i in range(20):
    plt.imshow(pred[i].reshape(256,-1)+0.3*np.array(imgs[primera+i, 0]))
    plt.title('prediccion, slce: {}'.format(primera+i))
    plt.show()
    plt.imshow(mask[primera+i]+0.3*np.array(imgs[primera+i, 0]))
    plt.title('etiqueta, slce: {}'.format(primera+i))
    plt.show()